In [1]:
import requests
import csv 
from collections import Counter
from pandas import *

In [2]:
listoffiles=[]
listoffilmnames=[]
        
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False 
    
def csvgetter(user,password,*otherusernames):
    #first create a session 's'
    s=requests.session()
    #this is the data that is sent to login to icheckmovies
    login_data={'login[username]': user,'login[password]': password,'submit': 'login',}
    #here we send the login data, using the session created before
    s.post("https://www.icheckmovies.com/login/", data=login_data) 
    #once again, whithin the session we get the csv link and call it 'r'
    usernames_list=list(otherusernames)
    usernames_list.append(user)
    for item in usernames_list:
        url='https://www.icheckmovies.com/movies/checked/?user='+item+'&export'
        r=s.get(url)
        checklist=r.text
        checklist_encode = checklist.encode('utf-8')
        filename=item+'_checklist'+'.csv'
        file=open(filename, "w")
        file.write(checklist_encode)
        file.close() 
        listoffiles.append(filename)
    numberofusers=str((len(listoffiles)))
    print('There are '+numberofusers+' users in the database: '+'\n'+'\n'.join(listoffiles))
    
def listmaker(username):
    filmlist=[]
    toplist=[]
    mostchecked=[]
    csvname=username+"_checklist.csv"
    csv_reader = csv.reader(open(csvname, "rb"), delimiter=';')
    for row in csv_reader:
        filmlist.append(row[0])
        mostcheckednum=row[3]
        if is_number(mostcheckednum) is True:
            mostcheckednum=int(row[3])
        mostchecked.append(mostcheckednum)
        toplistnum=row[5]
        if is_number(toplistnum) is True:
            toplistnum=int(row[5])
        toplist.append(toplistnum)        
    filmlist.pop(0)
    mostchecked.pop(0)
    toplist.pop(0)
    compiled_dataframe={'title' : filmlist,
            'most_checked': mostchecked,
            'top_lists': toplist}
    userlist=DataFrame(compiled_dataframe)
    return(userlist)

def multiname(*usernames):
    for name in usernames:
        listmaker(name)
        listoffilmnames.append(filmlist)
        print(name)

def process_club_list(user_list,others):
    merge_frames = others.merge(user_list,on=['title','most_checked'],how='left')
    not_same_titles = merge_frames[merge_frames.top_lists_y.isnull()]
    del not_same_titles['top_lists_y']
    not_same_titles.columns = ['most_checked', 'title', 'top_lists']
    return(not_same_titles)

#this is working on a function that will do everything
def Listmaker_pro(usr1,pssword,*usrs):
    username1=usr1
    username2=usrs[0]
    username3=usrs[1]
    password=pssword
    csvgetter(username1,password,username2,username3)
    user1_list=listmaker(username1)
    user2_list=listmaker(username2)
    user3_list=listmaker(username3)
    combo_1_2=user1_list.append(user2_list)
    seenby_1_2=combo_1_2[combo_1_2.duplicated('title')==True]
    combo_1_3=user1_list.append(user3_list)
    seenby_1_3=combo_1_3[combo_1_3.duplicated('title')==True]
    combo_2_3=user2_list.append(user3_list)
    seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==True] 
    club1_list = process_club_list(user1_list,seenby_2_3) 
    club2_list = process_club_list(user2_list,seenby_1_3)
    club3_list = process_club_list(user3_list,seenby_1_2)

In [3]:
username1='hermesmovies'
username2='Haessar'
username3='James88'
password=''

In [4]:
csvgetter(username1,password,username2,username3)

There are 3 users in the database: 
Haessar_checklist.csv
James88_checklist.csv
hermesmovies_checklist.csv


In [5]:
user1_list=listmaker(username1)
user2_list=listmaker(username2)
user3_list=listmaker(username3)
print("user1 is "+username1+", user2 is "+username2+" and user3 is "+username3+", user#_list is a dataframe of each users list")

user1 is hermesmovies, user2 is Haessar and user3 is James88, user#_list is a dataframe of each users list


Want to make a list that both james and will have seen that I haven't seen

In [6]:
#join User 1 and User 2
combo_1_2=user1_list.append(user2_list)
#return duplicates
seenby_1_2=combo_1_2[combo_1_2.duplicated('title')==True]

#join User 1 and User 3
combo_1_3=user1_list.append(user3_list)
seenby_1_3=combo_1_3[combo_1_3.duplicated('title')==True]

#join User 2 and User 3
combo_2_3=user2_list.append(user3_list)
seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==True] 



In [17]:
#Club list for User 1:
club1_list = process_club_list(user1_list,seenby_2_3) 

#Club list for User 2:
club2_list = process_club_list(user2_list,seenby_1_3)

#Club list for User 3:
club3_list = process_club_list(user3_list,seenby_1_2)

    most_checked                                            title  \
0          10351                                   The Wicker Man   
1          10351                                   The Wicker Man   
2          56833                                           Psycho   
3          56833                                           Psycho   
4          24771                                 Dawn of the Dead   
5          52869                              Alice in Wonderland   
6          93928                                               Up   
7          20367                                          Traffic   
8          57854                           The Godfather: Part II   
9          94244                                  The Incredibles   
10         17279              The Hobbit: The Desolation of Smaug   
11         31511                                       Annie Hall   
12         38152                                          Amadeus   
13         96188                  

In [15]:
club2_list.sort(['top_lists','most_checked'],ascending=False)

,most_checked,title,top_lists
17,56298,Léon,18
19,31591,Magnolia,16
297,54866,Full Metal Jacket,15
18,50502,Rain Man,14
148,50463,Twelve Monkeys,14
145,30080,RoboCop,12
224,23145,Gravity,11
210,49412,Pretty Woman,7
166,9283,Antichrist,5
152,9140,Election,5
